In [1]:
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import sqlite3

In [4]:
df=pd.read_csv('/content/drive/MyDrive/WalmartCommerce.csv', delimiter=';')

In [5]:
df.head(10)

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,vat,total,dtme,tme,payment_method,cogs,gross_margin_pct,gross_income,rating,time_of_day,day_name,month_name
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05 00:00:00,13:08:00,Ewallet,522.83,4.7619,26.1415,9.1,afternoon,Saturday,January
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08 00:00:00,10:29:00,Cash,76.40,4.7619,3.8200,9.6,morning,Friday,March
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03 00:00:00,13:23:00,Credit card,324.31,4.7619,16.2155,7.4,afternoon,Sunday,March
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27 00:00:00,20:33:00,Ewallet,465.76,4.7619,23.2880,8.4,evening,Sunday,January
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08 00:00:00,10:37:00,Ewallet,604.17,4.7619,30.2085,5.3,morning,Friday,February
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,2019-03-25 00:00:00,18:30:00,Ewallet,597.73,4.7619,29.8865,4.1,evening,Monday,March
6,355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.6520,433.6920,2019-02-25 00:00:00,14:36:00,Ewallet,413.04,4.7619,20.6520,5.8,afternoon,Monday,February
7,315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.56,10,36.7800,772.3800,2019-02-24 00:00:00,11:38:00,Ewallet,735.60,4.7619,36.7800,8.0,morning,Sunday,February
8,665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.6260,76.1460,2019-01-10 00:00:00,17:15:00,Credit card,72.52,4.7619,3.6260,7.2,evening,Thursday,January
9,692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.84,3,8.2260,172.7460,2019-02-20 00:00:00,13:27:00,Credit card,164.52,4.7619,8.2260,5.9,afternoon,Wednesday,February


In [6]:
df.isnull().sum(axis = 0)

invoice_id          0
branch              0
city                0
customer_type       0
gender              0
product_line        0
unit_price          0
quantity            0
vat                 0
total               0
dtme                0
tme                 0
payment_method      0
cogs                0
gross_margin_pct    0
gross_income        0
rating              0
time_of_day         0
day_name            0
month_name          0
dtype: int64

In [20]:
# Connect to SQLite database (creates a new database if it doesn't exist)
conn = sqlite3.connect('Retail.sqlite')

# Use the DataFrame's to_sql() method to create a table and insert data
df.to_sql(name='my_table', con=conn, index=False, if_exists='replace')

# Commit the changes
conn.commit()

# Query the data from the created table
result = pd.read_sql('SELECT * FROM my_table', conn)

# Display the result
print(result)

      invoice_id branch       city customer_type  gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               product_line  unit_price  quantity      vat      total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyle       46.33         7  16.2155  

In [8]:
# Monthly Revenue, quantity sold, Profits and Customer Satisfaction
query= "select month_name, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df1=pd.read_sql(query, conn)
df1

,month_name,Revenue,Quantity_Sold,Rating,Total_Profit
0,January,116291.868,1965,7.017614,5537.708
1,March,109455.507,1891,6.840290,5212.167
2,February,97219.374,1654,7.071287,4629.494


In [9]:
# City wise number of Transactions
query="select city, count(distinct invoice_id) as NoOfTransactions from my_table group by 1 order by 2 desc"

df2=pd.read_sql(query, conn)
df2

,city,NoOfTransactions
0,Yangon,340
1,Mandalay,332
2,Naypyitaw,328


In [17]:
# Product Line vs Quantity Sold, Revenue, Profit, Rating
query= "select product_line, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df3=pd.read_sql(query, conn)
df3

,product_line,Revenue,Quantity_Sold,Rating,Total_Profit
0,Food and beverages,56144.8440,952,7.113218,2673.5640
1,Sports and travel,55122.8265,920,6.916265,2624.8965
2,Electronic accessories,54337.5315,971,6.924706,2587.5015
3,Fashion accessories,54305.8950,902,7.029213,2585.9950
4,Home and lifestyle,53861.9130,911,6.837500,2564.8530
5,Health and beauty,49193.7390,854,7.003289,2342.5590


In [11]:
# Payment Method Vs Revenue, Profit, Quantity Sold, Rating
query= "select payment_method, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df4=pd.read_sql(query, conn)
df4

,payment_method,Revenue,Quantity_Sold,Rating,Total_Profit
0,Cash,112206.570,1896,6.970058,5343.170
1,Ewallet,109993.107,1892,6.947826,5237.767
2,Credit card,100767.072,1722,7.003215,4798.432


In [12]:
# Customer Type vs Revenue, Profit, Quantity Sold, Rating
query= "select customer_type, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df5=pd.read_sql(query, conn)
df5

,customer_type,Revenue,Quantity_Sold,Rating,Total_Profit
0,Member,164223.444,2785,6.940319,7820.164
1,Normal,158743.305,2725,7.005210,7559.205


In [13]:
# Gender Vs Revenue, Profit, Quantity Sold, Rating
query= "select gender, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df6=pd.read_sql(query, conn)
df6

,gender,Revenue,Quantity_Sold,Rating,Total_Profit
0,Female,167882.925,2869,6.964471,7994.425
1,Male,155083.824,2641,6.980962,7384.944


In [23]:
# Citywise Average Profit per Transaction
query= "select city, avg(total-cogs) as Average_Profit from my_table group by 1"

df7=pd.read_sql(query, conn)
df7

,city,Average_Profit
0,Mandalay,15.232024
1,Naypyitaw,16.052367
2,Yangon,14.874001


In [25]:
# Net Average profit per Transaction
query= "select avg(total-cogs) as Average_Profit from my_table"

df8=pd.read_sql(query, conn)
df8

,Average_Profit
0,15.379369


In [26]:
# Total Sales vs Time of Day
query= "select time_of_day, sum(quantity) as Quantity_Sold from my_table group by 1 order by 2 desc"

df9=pd.read_sql(query, conn)
df9

,time_of_day,Quantity_Sold
0,evening,2361
1,afternoon,2111
2,morning,1038


In [27]:
# Branchwise Revenue, Profit, Quantity Sold, Rating
query= "select branch, sum(total) as Revenue, sum(quantity) as Quantity_Sold, avg(rating) as Rating, sum(total-cogs) as Total_Profit from my_table  group by 1 order by 2 desc"

df10=pd.read_sql(query, conn)
df10

,branch,Revenue,Quantity_Sold,Rating,Total_Profit
0,C,110568.7065,1831,7.072866,5265.1765
1,A,106200.3705,1859,7.027059,5057.1605
2,B,106197.6720,1820,6.818072,5057.0320


In [28]:
# Average Rating
query= "select avg(rating) from my_table"

df11=pd.read_sql(query, conn)
df11

,avg(rating)
0,6.9727


In [18]:
conn.close()